### Building chatbot with multiple tools using langgraph

### Create a chatbot with tool capabilities with arxiv, wikipedia search and some functions

In [26]:
from typing_extensions import TypedDict
from langgraph.graph import StateGraph,START,END

In [27]:
import os
from dotenv import load_dotenv
load_dotenv()
os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")
os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")
os.environ["TAVILY_API_KEY"]=os.getenv("TAVILY_API_KEY")

In [28]:
from langchain_groq import ChatGroq
llm_groq=ChatGroq(model="llama-3.3-70b-versatile")
llm_groq.invoke("hi I am sachchida. I am AI enthusiast")

AIMessage(content="Hello Sachchida, nice to meet you. It's great to hear that you're an AI enthusiast. There have been so many advancements in the field of Artificial Intelligence recently, from natural language processing to computer vision. What specific areas of AI interest you the most? Are you exploring any AI-related projects or learning about AI through courses or books?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 71, 'prompt_tokens': 46, 'total_tokens': 117, 'completion_time': 0.215613866, 'prompt_time': 0.002251057, 'queue_time': 0.050442942, 'total_time': 0.217864923}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_34d416ee39', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--ee4acaf2-02be-4bcc-b9e8-2cfad41d6242-0', usage_metadata={'input_tokens': 46, 'output_tokens': 71, 'total_tokens': 117})

In [29]:
from langchain_community.tools import ArxivQueryRun,WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper, ArxivAPIWrapper

In [30]:
api_wrapper_arxiv=ArxivAPIWrapper(top_k_results=2, doc_content_chars_max=500)
arxiv=ArxivQueryRun(api_wrapper=api_wrapper_arxiv)
print(arxiv.name)

arxiv


In [31]:
arxiv.invoke("Attention is all you need")

"Published: 2024-07-22\nTitle: Attention Is All You Need But You Don't Need All Of It For Inference of Large Language Models\nAuthors: Georgy Tyukin, Gbetondji J-S Dovonon, Jean Kaddour, Pasquale Minervini\nSummary: The inference demand for LLMs has skyrocketed in recent months, and serving\nmodels with low latencies remains challenging due to the quadratic input length\ncomplexity of the attention layers. In this work, we investigate the effect of\ndropping MLP and attention layers at inference time o"

In [32]:
api_wrapper_wiki=WikipediaAPIWrapper(top_k_results=2, doc_content_chars_max=500)
wiki=WikipediaQueryRun(api_wrapper=api_wrapper_wiki)
print(wiki.name)

wikipedia


In [33]:
wiki.invoke("barack obama")

'Page: Barack Obama\nSummary: Barack Hussein Obama II (born August 4, 1961) is an American politician who served as the 44th president of the United States from 2009 to 2017. A member of the Democratic Party, he was the first African American president. Obama previously served as a U.S. senator representing Illinois from 2005 to 2008 and as an Illinois state senator from 1997 to 2004.\nBorn in Honolulu, Hawaii, Obama graduated from Columbia University in 1983 with a Bachelor of Arts degree in polit'

In [34]:
from langchain_community.tools.tavily_search import TavilySearchResults
tavily=TavilySearchResults()

In [35]:
tavily.invoke("provide the current USD to INR conversion rate")

[{'title': '1 USD to INR - US Dollars to Indian Rupees Exchange Rate - Xe',
  'url': 'https://www.xe.com/en-us/currencyconverter/convert/?Amount=1&From=USD&To=INR',
  'content': '# 1 USD to INR - Convert US Dollars to Indian Rupees\n\nXe Currency Converter\n\nusd\ninr\n\n1.00 US Dollar =\n\n87.91523 Indian Rupee\n\n1 INR = 0.0113746 USD\n\n### Looking to make large transfers?\n\nWe can beat competitor rates\n\nWe use the mid-market rate for our Converter. This is for informational purposes only. You won’t receive this rate when sending money. Login to view send rates\n\n#### Convert US Dollar to Indian Rupee\n\nRate information of USD/INR currency pair [...] | inr Indian RupeeINR | usd US DollarUSD |\n --- |\n| 1 INR | 0.0113746 USD |\n| 5 INR | 0.056873 USD |\n| 10 INR | 0.113746 USD |\n| 25 INR | 0.284365 USD |\n| 50 INR | 0.56873 USD |\n| 100 INR | 1.13746 USD |\n| 500 INR | 5.6873 USD |\n| 1,000 INR | 11.3746 USD |\n| 5,000 INR | 56.873 USD |\n| 10,000 INR | 113.746 USD |\n\ninr\nu

In [36]:
### combine all the tools in the list
tools=[arxiv,wiki,tavily]

In [37]:
### intialize my llm model
from langchain_groq import ChatGroq
llm=ChatGroq(model="openai/gpt-oss-20b")
### bind llm with tools (list of tools)
llm_with_tools=llm.bind_tools(tools)

In [38]:
from langchain_core.messages import AIMessage, HumanMessage
from pprint import pprint

In [39]:
llm_with_tools.invoke([HumanMessage(content=f"what is the recent AI news")])

AIMessage(content='', additional_kwargs={'reasoning_content': 'User asks: "what is the recent AI news". Likely they want a summary of recent AI news. We should browse current news. Use tavily_search_results_json for up-to-date info. Let\'s search.', 'tool_calls': [{'id': 'fc_1f2e74d0-7ade-493a-8050-4e1b9a411181', 'function': {'arguments': '{"query":"recent AI news 2025 October"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 77, 'prompt_tokens': 291, 'total_tokens': 368, 'completion_time': 0.075824356, 'prompt_time': 0.015590051, 'queue_time': 0.045156424, 'total_time': 0.091414407}, 'model_name': 'openai/gpt-oss-20b', 'system_fingerprint': 'fp_77f8660d1d', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--9c510d69-a660-47fa-87b9-588b0035b71e-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'recent AI news 2025 October'}, 'id': 'fc_1f2e74d0-7ade-493a-80

In [40]:
llm_with_tools.invoke([HumanMessage(content=f"what is the recent AI news")]).tool_calls

[{'name': 'tavily_search_results_json',
  'args': {'query': 'recent AI news'},
  'id': 'fc_d19738ed-44d6-404d-9026-ebde64e46091',
  'type': 'tool_call'}]

In [45]:
### Entire chatbot with langgraph
from IPython.display import Image, display
from langgraph.graph import StateGraph, START,END
from langgraph.prebuilt import ToolNode
from langgraph.prebuilt import tools_condition

In [46]:
from typing_extensions import TypedDict
from langchain_core.messages import AnyMessage
from typing import Annotated
from langgraph.graph.message import add_messages
class State(TypedDict):
    messages:Annotated[list[AnyMessage],add_messages]

In [47]:
### Node definition
def tool_calling_llm(state:State):
    return {"messages":[llm_with_tools.invoke(state["messages"])]}

In [51]:
builder=StateGraph(State)
builder.add_node("tool_calling_llm",tool_calling_llm)
builder.add_node("tools",ToolNode(tools))
builder.add_edge(START,"tool_calling_llm")

builder.add_conditional_edges(
    "tool_calling_llm",
    tools_condition
)
builder.add_edge("tools",END)
graph=builder.compile()

In [53]:
from pprint import pprint, PrettyPrinter

In [56]:
messages=graph.invoke({"messages":HumanMessage(content="what is attention is all you need")})
for m in messages['messages']:
    m.pretty_print()

================================ Human Message =================================

what is attention is all you need
================================== Ai Message ==================================
Tool Calls:
  tavily_search_results_json (fc_b46bf656-92b2-4343-a235-3fa3acbd033e)
 Call ID: fc_b46bf656-92b2-4343-a235-3fa3acbd033e
  Args:
    query: Attention Is All You Need paper summary
================================= Tool Message =================================
Name: tavily_search_results_json

[{"title": "“Attention is All You Need” Summary - Medium", "url": "https://medium.com/@dminhk/attention-is-all-you-need-summary-6f0437e63a91", "content": "Sitemap\n\nOpen in app\n\nSign in\n\nSign in\n\n# “Attention is All You Need” Summary\n\nDavid Min\n\n3 min readAug 14, 2023\n\n> Source: \n\n“Attention Is All You Need” is a research paper by Ashish Vaswani et al. that proposes a new neural network architecture for sequence-to-sequence tasks, called the Transformer model. The paper challe

In [58]:
messages=graph.invoke({"messages":HumanMessage(content="1706.03762")})
for m in messages['messages']:
    m.pretty_print()

================================ Human Message =================================

1706.03762
================================== Ai Message ==================================
Tool Calls:
  arxiv (fc_d925b6d4-74c7-4b97-bda2-a5e698eba033)
 Call ID: fc_d925b6d4-74c7-4b97-bda2-a5e698eba033
  Args:
    query: 1706.03762
================================= Tool Message =================================
Name: arxiv

Published: 2023-08-02
Title: Attention Is All You Need
Authors: Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz Kaiser, Illia Polosukhin
Summary: The dominant sequence transduction models are based on complex recurrent or
convolutional neural networks in an encoder-decoder configuration. The best
performing models also connect the encoder and decoder through an attention
mechanism. We propose a new simple network architecture, the Transformer, base


In [59]:
messages=graph.invoke({"messages":HumanMessage(content="who is barrack obama")})
for m in messages['messages']:
    m.pretty_print()

================================ Human Message =================================

who is barrack obama
================================== Ai Message ==================================
Tool Calls:
  wikipedia (fc_5a3d0da8-1301-423e-ad8e-d500a26cb687)
 Call ID: fc_5a3d0da8-1301-423e-ad8e-d500a26cb687
  Args:
    query: Barack Obama
================================= Tool Message =================================
Name: wikipedia

Page: Barack Obama
Summary: Barack Hussein Obama II (born August 4, 1961) is an American politician who served as the 44th president of the United States from 2009 to 2017. A member of the Democratic Party, he was the first African American president. Obama previously served as a U.S. senator representing Illinois from 2005 to 2008 and as an Illinois state senator from 1997 to 2004.
Born in Honolulu, Hawaii, Obama graduated from Columbia University in 1983 with a Bachelor of Arts degree in polit
